# 🧠 Extracción de Características

## 📝 Descripción
Este notebook corresponde a la **segunda fase del pipeline** para la detección de imágenes médicas manipuladas. Aquí se realiza la **extracción de características relevantes** a partir de las imágenes procesadas y normalizadas en la fase anterior. Estas características serán usadas como entrada para los modelos de clasificación binaria.  
Se implementan técnicas clásicas de procesamiento de imágenes para capturar información sobre la textura, bordes y patrones presentes en las imágenes.  

---

## 🎯 Objetivos
✅ Leer las imágenes procesadas desde la carpeta `experiments_png/`.  
✅ Extraer descriptores de características como:  
- Gray Level Co-occurrence Matrix (GLCM).  
- Histogram of Oriented Gradients (HOG).  
- Local Binary Patterns (LBP).  
- Color spaces (LAB).  
✅ Explorar otras posibles técnicas como SIFT o SURF (*si el dominio de la imagen lo permite*).  
✅ Guardar las características extraídas en un formato tabular para su uso en la fase de entrenamiento.

---

## 📦 📤 Outputs de la Fase
Al finalizar esta fase se generan los siguientes elementos:

📄 **`features_extracted.csv`**  
Archivo CSV que contiene el conjunto de características extraídas para cada imagen. Incluye:  
- Identificador de la imagen.  
- Vector de características (GLCM, LBP, etc.).  
- Clase objetivo (manipulada / no manipulada).  

In [28]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import local_binary_pattern
from skimage.feature import graycomatrix, graycoprops
from skimage.color import rgb2gray
from tqdm import tqdm  # Barra de progreso
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import ResNet18_Weights
from PIL import Image

In [29]:
def extraer_features_glcm(img):
    # Configuración de GLCM
    distances = [1]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]

    # ------ GLCM Features ------ #
    glcm = graycomatrix(img, 
                        distances=distances, 
                        angles=angles, 
                        levels=256, 
                        symmetric=True, 
                        normed=True)
    
    contraste = graycoprops(glcm, 'contrast').flatten()
    homogeneidad = graycoprops(glcm, 'homogeneity').flatten()
    energia = graycoprops(glcm, 'energy').flatten()
    correlacion = graycoprops(glcm, 'correlation').flatten()
    entropia = -np.sum(glcm * np.log2(glcm + 1e-10), axis=(0, 1)).flatten()

    return np.concatenate([contraste, homogeneidad, energia, correlacion, entropia])

def extraer_features_lbp(img):
     # Configuración de LBP
    radius = 1
    n_points = 8 * radius
    method = 'uniform'  # Da un histograma de 59 bins con 8 vecinos

    lbp = local_binary_pattern(img, n_points, radius, method)
    hist, _ = np.histogram(lbp.ravel(), 
                           bins=np.arange(0, n_points + 3),
                           range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)  # Normalización

    features_lbp = hist

def extraer_features_hog(img):

    win_size = (128, 128)
    block_size = (32, 32)
    block_stride = (16, 16)
    cell_size = (8, 8)
    nbins = 9

    image = cv2.resize(img, win_size)

    hog = cv2.HOGDescriptor( win_size,
                             block_size,
                             block_stride,
                             cell_size,
                             nbins)
    features_hog = hog.compute(img).flatten()
    return features_hog

In [31]:
# ResNet preentrenada
 
resnet = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)# Puedes usar resnet34, resnet50, etc.
resnet.eval()  # Modo evaluación

# Quitar la capa final (clasificación) para quedarte solo con los features
feature_extractor = torch.nn.Sequential(*list(resnet.children())[:-1])

# Transformaciones necesarias para la imagen
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Las redes esperan 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImagenNet stats
                         std=[0.229, 0.224, 0.225])
])

# ---------------- Función para extraer features ---------------- #

def extraer_features_resnet(img):
    img = Image.fromarray(img).convert('RGB')
    img = transform(img).unsqueeze(0)  # Añadir batch dimension
    with torch.no_grad():
        features = feature_extractor(img).squeeze().numpy()  # shape: (512,)
    return features

In [39]:


# ---------------- Configuración ---------------- #
carpeta_png = 'Experiments-png/'  # Carpeta donde están los PNGs
labels = pd.read_csv('labels_final.csv')  # Tu dataframe con columnas uuid, slice, label

# Lista para guardar resultados
lista_features = []

# ---------------- Recorrido ---------------- #
for idx, row in tqdm(labels.iterrows(), total=labels.shape[0]):
    uuid = row['uuid']
    slice_num = row['slice']
    etiqueta = row['label']

    ruta_imagen = os.path.join(carpeta_png, f"{uuid}/{slice_num}.png")
    
    if not os.path.exists(ruta_imagen):
        print(f"Advertencia: No se encontró {ruta_imagen}")
        continue

    img = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Error cargando {ruta_imagen}")
        continue
    
    # Extraer features
    features_concatenados = []
    columnas = []

    features_resnet = extraer_features_resnet(img)
    features_concatenados = np.concatenate([features_concatenados, features_resnet])
    columnas = np.concatenate([columnas,[f'resnet_feat_{i}' for i in range(features_resnet.shape[0])]])
    '''
    features_lbp = extraer_features_lbp(img)
    features_concatenados = np.concatenate([features_concatenados, features_lbp])
    columnas = np.concatenate([columnas,[f'lbp_hist_{i}' for i in range(features_lbp.shape[0])]])

    features_hog = extraer_features_hog(img)
    features_concatenados = np.concatenate([features_concatenados, features_hog])
    columnas = np.concatenate([columnas,[f'hog_feat_{i}' for i in range(features_hog.shape[0])]])

    features_glcm = extraer_features_glcm(img)
    features_glcm = features_glcm.flatten()  # Asegurarse de que es un vector 1D
    columnas = np.concatenate([columnas,[f'glcm_feat_{i}' for i in range(features_glcm.shape[0])]])
    '''

    # Guardar en lista con uuid, slice y label
    lista_features.append([uuid, slice_num, etiqueta] + features_concatenados.tolist())

# ---------------- DataFrame final ---------------- #

# Si tus columnas son un array de nombres, conviértelo a lista plana
if isinstance(columnas, np.ndarray):
    columnas = columnas.tolist()

# Si tus features son: uuid, slice, label, feat1, feat2, ..., featN
# Asegúrate de que columnas es una lista plana de nombres de columnas de features
columnas_finales = ['uuid', 'slice', 'label'] + columnas
df_features = pd.DataFrame(lista_features, columns=columnas_finales)

# Eliminar columna slice si no la necesitas
df_features = df_features.drop(columns=['slice'])

# Reordenar columnas para poner uuid y label al final si así lo deseas
columnas_ordenadas = [col for col in df_features.columns if col not in ['uuid', 'label']] + ['uuid', 'label']
df_features = df_features[columnas_ordenadas]

100%|██████████| 492/492 [00:10<00:00, 47.26it/s]


In [ ]:
X = np.array(df_features.drop(columns=['uuid', 'label']))
y = np.array(df_features['label'])
groups = np.array(df_features['uuid'])

print(f"✅ Shape final X: {X.shape}")
print(f"✅ Shape final y: {y.shape}")
print(f"✅ Shape final groups: {groups.shape}")

np.savez_compressed("data.npz", X=X, y=y, groups=groups)
print("✅ Archivo 'data.npz' guardado exitosamente.")

✅ Shape final X: (492, 512)
✅ Shape final y: (492,)
✅ Shape final groups: (492,)
✅ Archivo 'features_labels_resnet.npz' guardado exitosamente.
